In [9]:
from pymongo import MongoClient
from semanticscholar import SemanticScholar
from tqdm import tqdm

In [10]:
client = MongoClient("mongodb://127.0.0.1:27017/")
db = client["semantic_scholar"]

In [11]:
collection = db["papers"]

In [13]:
params = {
    'query': '',
    'fields': [], # [] if all
    'fields_of_study': [], # [] if all
    'year': None, # None if all
    'limit': 100
}

In [14]:
def get_paper_overalls(sch, query, year, limit=100, fields_of_study=None, fields=None):
    local_params = {
        'query': query,
        'limit': limit
    }
    if limit >= 100:
        local_params['limit'] = 100
    if year is not None:
        local_params['year'] = year
    if fields_of_study is not None:
        local_params['fields_of_study'] = fields_of_study
    if fields is not None:
        local_params['fields'] = fields
    
    results = sch.search_paper(**local_params)
    return [results, len(results)]

In [15]:
def main(local_params, how_many=250):
    paper_overalls = list()
    sch = SemanticScholar()
    sch.timeout = round(how_many*0.0625 + 1) + 2
    package = get_paper_overalls(sch=sch, **local_params)
    sch_iter = package[0]
    content = sch_iter.items
    paper_overalls.append(content)
    count = package[1]
    with tqdm(total=how_many) as pbar:
        pbar.update(count)
        if count < how_many:
            while count < how_many:
                try:
                    sch_iter.next_page()
                    content = sch_iter.items
                    if type(content[0]) != list:
                        paper_overalls.append(content[0])
                    else:
                        paper_overalls += content[0]
                    count += package[1]
                    pbar.update(package[1])
                except Exception as e:
                    print(e)
                    print(package[0])
                    break
    return papers_to_dicts(paper_overalls[0])

In [16]:
def papers_to_dicts(p_list):
    d_list = list()
    for p in p_list:
        p_dict =p.__dict__['_data']
        d_list.append(p_dict)

    return d_list

In [19]:
ai_topics = [
    "Genetics and AI", "Physics and AI", "Neuroscience and AI", "Astronomy and AI",
    "Chemistry and AI", "Ecology and AI", "Mathematics and AI", "Geology and AI",
    "Bioinformatics and AI", "Psychology and AI", "Biology and AI", "Anthropology and AI",
    "Medicine and AI", "Sociology and AI", "Economics and AI", "Linguistics and AI",
    "History and AI", "Geography and AI", "Engineering and AI", "Cybersecurity and AI",
    "Robotics and AI", "Genomics and AI", "Climatology and AI", "Finance and AI",
    "Literary Studies and AI", "Philosophy and AI", "Archaeology and AI", "Nanotechnology and AI",
    "Political Science and AI", "Music and AI", "Gender Studies and AI", "Cosmology and AI",
    "Pedagogy and AI", "Informatics and AI", "Ethology and AI", "Marketing and AI",
    "Pharmacology and AI", "Rhetoric and AI", "Theology and AI", "Design and AI",
    "Psychiatry and AI", "Architecture and AI", "Jurisprudence and AI", "Genetic Engineering and AI",
    "Psychophysiology and AI", "Astrophysics and AI", "Econometrics and AI", "Cultural Studies and AI",
    "Biochemistry and AI", "Sports Sciences and AI"
]

In [20]:
result_all = list()

In [21]:
for topic in ai_topics:
    params['query'] = topic
    result = main(params, how_many=1000)
    result_all += result

 30%|███       | 300/1000 [00:04<00:09, 74.94it/s]


No more pages to fetch.


 90%|█████████ | 900/1000 [02:39<00:17,  5.64it/s]


No more pages to fetch.


 90%|█████████ | 900/1000 [01:54<00:12,  7.87it/s]


...

No more pages to fetch.


 60%|██████    | 600/1000 [04:19<02:53,  2.31it/s]


No more pages to fetch.


 20%|██        | 200/1000 [00:32<02:11,  6.10it/s]

No more pages to fetch.


In [22]:
len(result_all)

35246

In [23]:
collection.insert_many(result_all)

InsertManyResult([ObjectId('664cc750e916764cf4cc72aa'), ObjectId('664cc750e916764cf4cc72ab'), ObjectId('664cc750e916764cf4cc72ac'), ObjectId('664cc750e916764cf4cc72ad'), ObjectId('664cc750e916764cf4cc72ae'), ObjectId('664cc750e916764cf4cc72af'), ObjectId('664cc750e916764cf4cc72b0'), ObjectId('664cc750e916764cf4cc72b1'), ObjectId('664cc750e916764cf4cc72b2'), ObjectId('664cc750e916764cf4cc72b3'), ObjectId('664cc750e916764cf4cc72b4'), ObjectId('664cc750e916764cf4cc72b5'), ObjectId('664cc750e916764cf4cc72b6'), ObjectId('664cc750e916764cf4cc72b7'), ObjectId('664cc750e916764cf4cc72b8'), ObjectId('664cc750e916764cf4cc72b9'), ObjectId('664cc750e916764cf4cc72ba'), ObjectId('664cc750e916764cf4cc72bb'), ObjectId('664cc750e916764cf4cc72bc'), ObjectId('664cc750e916764cf4cc72bd'), ObjectId('664cc750e916764cf4cc72be'), ObjectId('664cc750e916764cf4cc72bf'), ObjectId('664cc750e916764cf4cc72c0'), ObjectId('664cc750e916764cf4cc72c1'), ObjectId('664cc750e916764cf4cc72c2'), ObjectId('664cc750e916764cf4cc72